In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import requests
import pandas as pd

dataset_url = "https://drive.google.com/uc?id=1slcvP4WTNOefZiaqf9z_EqveFSewJjsV&export=download"

response = requests.get(dataset_url)
content = response.content

with open("dataset.csv", "wb") as file:
    file.write(content)

df = pd.read_csv("dataset.csv")



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
df.shape

(50000, 2)

In [ ]:
#Count the no. of positive and negative review
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
!pip install fastapi uvicorn
!pip install pyngrok
!pip install fastapi nest-asyncio pyngrok uvicorn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=f551d211d22f5c6fc13f6a08947625428b69db567b2e8fbe5b847cd79c738996
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
import tensorflow_hub as hub
import nest_asyncio
import numpy as np
from typing import List
from pyngrok import ngrok

from fastapi import FastAPI
app=FastAPI()

In [ ]:
import pickle



# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

 # Define the batch size
batch_size = 1000

# Tokenize and encode the reviews in batches
x_batches = []
num_samples = len(df['review'])
num_batches = num_samples // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    reviews_batch = df['review'].iloc[start_idx:end_idx].tolist()
    embeddings_batch = embed(reviews_batch).numpy()
    x_batches.append(embeddings_batch)

# Process the remaining samples (if any) that don't fit in a full batch
remaining_samples = num_samples % batch_size
if remaining_samples > 0:
    start_idx = num_batches * batch_size
    reviews_batch = df['review'].iloc[start_idx:].tolist()
    embeddings_batch = embed(reviews_batch).numpy()
    x_batches.append(embeddings_batch)

# Concatenate the batches into a single numpy array
x = np.concatenate(x_batches, axis=0)

#Changing the sentiment values as 1 and 0 for positive and negative respectively
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(df['sentiment'])

#Splitting the data into the training and the testing set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

#Building and training the SVM model
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)

# Save the model and label encoder using pickle


model_path = "/content/trained_model.pickle"
encoder_path = "/content/label_encoder.pickle"

with open(model_path, "wb") as file:
    pickle.dump(model, file)

with open(encoder_path, "wb") as file:
    pickle.dump(le, file)





In [ ]:
import os

model_path = "/content/trained_model.pickle"
encoder_path = "/content/label_encoder.pickle"

# Check if the model file exists
if os.path.exists(model_path):
    print("Model file exists")
else:
    print("Model file does not exist")

# Check if the label encoder file exists
if os.path.exists(encoder_path):
    print("Label encoder file exists")
else:
    print("Label encoder file does not exist")


Model file exists
Label encoder file exists


In [ ]:
import pickle
import random
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

import os
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

nest_asyncio.apply()

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)

app=FastAPI()
# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["GET", "POST", "OPTIONS"],
    allow_headers=["Content-Type"],
)



class PredictRequest(BaseModel):
    review: str


@app.post("/predict")
async def predict_sentiment(request: PredictRequest):
    # Load the trained model and label encoder
    model_path = "/content/trained_model.pickle"
    encoder_path = "/content/label_encoder.pickle"

    with open(model_path, "rb") as file:
        model = pickle.load(file)

    with open(encoder_path, "rb") as file:
        label_encoder = pickle.load(file)

    # Load the Universal Sentence Encoder
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    # Encode the input review
    embedding = embed([request.review]).numpy()

    # Make prediction using the trained model
    prediction = model.predict(embedding)[0]

    # Convert the numerical prediction back to a sentiment label
    sentiment_label = label_encoder.inverse_transform([prediction])[0]

    # Return the predicted sentiment label as a response
    return {"prediction": sentiment_label}

if __name__ == "__main__":
   uvicorn.run(app, host="0.0.0.0", port = 8000)

Public URL: https://fdd6-34-68-68-214.ngrok.io


INFO:     Started server process [690]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     106.207.92.182:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.207.92.182:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.207.92.182:0 - "POST /predict HTTP/1.1" 200 OK
